In [213]:
#
# classifer notebook
#
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#
class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    
#
# PARAMS
#

batch_size = 4

input_size = 2 * (21 * 3) + 12 #138
hidden_size = 96
num_classes = 3

learning_rate = .01
num_epochs = 100


#
# INITIAL DATA
#

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/landmarks_indexed.csv",
                              transform=transformations)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#for name, param in model.named_parameters():
#    if param.requires_grad:
#        print(name, param.data)

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        
        running_loss_epoch += loss.item()
        if (batch_idx+1) % len(dataloader) == 0: #NB len(dataloader) is num of batches
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0



print("Done")

Epoch [1/100], Loss: 1.0020, Running Loss: 10.1818
Epoch [2/100], Loss: 0.8573, Running Loss: 7.9310
Epoch [3/100], Loss: 1.1827, Running Loss: 8.0608
Epoch [4/100], Loss: 1.4946, Running Loss: 8.6303
Epoch [5/100], Loss: 0.8845, Running Loss: 7.3328
Epoch [6/100], Loss: 1.1957, Running Loss: 7.8799
Epoch [7/100], Loss: 1.3213, Running Loss: 7.9595
Epoch [8/100], Loss: 1.0607, Running Loss: 7.5197
Epoch [9/100], Loss: 1.1359, Running Loss: 7.6812
Epoch [10/100], Loss: 1.1595, Running Loss: 7.4898
Epoch [11/100], Loss: 1.1035, Running Loss: 7.3037
Epoch [12/100], Loss: 1.3628, Running Loss: 7.5003
Epoch [13/100], Loss: 0.8417, Running Loss: 7.1274
Epoch [14/100], Loss: 1.0260, Running Loss: 7.3442
Epoch [15/100], Loss: 0.5787, Running Loss: 6.8620
Epoch [16/100], Loss: 1.3733, Running Loss: 7.3900
Epoch [17/100], Loss: 0.9458, Running Loss: 6.8751
Epoch [18/100], Loss: 1.4006, Running Loss: 7.1795
Epoch [19/100], Loss: 0.9328, Running Loss: 7.0055
Epoch [20/100], Loss: 0.8103, Running L

In [214]:
#
# Save / Load test
#
torch.save(model.state_dict(), './model.pt')
model.load_state_dict(torch.load('./model_overfit.pt'))

<All keys matched successfully>

In [215]:
#
# EVAL
#

accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        #print(out.data)
        _, klass = torch.max(out.data, 1)
        #print("Shape: ", landmarks.shape)
        #print("M", labels, klass, _)
        #print()
        print(klass, labels, klass==labels)
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))

print('--------')

#
# Exercise with batch dims
# so NN module *defaults* to batch size B x input - build to expect a batch
#
# unsqueeze adds a batch dim of size 1 and 0th place
# .item() is scalar value of a 1D tensor
#
import random
idx = random.choice( range(0, len(dataset)))
label, landmarks = dataset[idx]
#print(idx, label, landmarks)
#print("Shape: ", landmarks.shape)
pseudo_batch_landmarks = landmarks.unsqueeze(0)
#print("Shape unsqueeze: ", pseudo_batch_landmarks.shape)
#print(idx, label, pseudo_batch_landmarks)

out = model(pseudo_batch_landmarks)
_, klass = torch.max(out.data, 1)
print("Guess: ", klass, "Label: ", label)
print("Data: ", out.data)

tensor([2, 2, 0, 0]) tensor([2, 2, 0, 0]) tensor([True, True, True, True])
tensor([1, 0, 1, 2]) tensor([1, 0, 1, 2]) tensor([True, True, True, True])
tensor([1, 0, 2, 0]) tensor([1, 0, 2, 0]) tensor([True, True, True, True])
tensor([2, 1, 1, 2]) tensor([2, 1, 1, 2]) tensor([True, True, True, True])
tensor([1, 2, 1, 1]) tensor([1, 2, 1, 1]) tensor([True, True, True, True])
tensor([1, 0, 0, 2]) tensor([1, 0, 0, 2]) tensor([True, True, True, True])
tensor([1]) tensor([1]) tensor([True])
--------
Accuracy 25/25 : 1.0000
--------
Guess:  tensor([1]) Label:  1
Data:  tensor([[-1.1774,  2.2743, -0.7910]])
